In [22]:
import pyscheduling.JS.JmCmax as jm_cmax
from pyscheduling.JS.JobShop import Job, JobShopSolution

from docplex.cp.model import CpoModel
from docplex.cp.solver.cpo_callback import CpoCallback
from docplex.cp.expression import INTERVAL_MAX

In [29]:
def csp_transform_solution(msol, M_k, types_k, instance ):

    sol = JobShopSolution(instance)
    for k in range(instance.m):
        k_tasks = []
        for i, oper_k_i in enumerate(M_k[k]):
            start, end, _ = msol[oper_k_i]
            k_tasks.append(Job(types_k[k][i],start,end))

            k_tasks = sorted(k_tasks, key= lambda x: x[1])
            sol.machines[k].job_schedule = k_tasks
        
    return sol

In [24]:
instance = jm_cmax.JmCmax_Instance.generate_random(20,4)
E = range(instance.n)
M = range(instance.m)

model = CpoModel("JS_Model")

E_i = [[] for i in E]
M_k = [[] for k in M]

types_k = [ [] for k in M ]
for i in E:
    for oper in instance.P[i]:
        k, p_ik = oper
        start_period = (0, INTERVAL_MAX)
        oper_i_k = model.interval_var( start = start_period,
                                       size = p_ik, optional= False, name=f'E[{i},{k}]')
        E_i[i].append(oper_i_k)
        
        M_k[k].append(oper_i_k)
        types_k[k].append(i)

# No overlap inside machines
seq_array = []
for k in M:
    seq_k = model.sequence_var(M_k[k], types_k[k], name=f"Seq_{k}")
    model.add( model.no_overlap(seq_k) )        
    seq_array.append(seq_k)

# Precedence constraint between machines for each job
for i in E:
    for k in range(1, len(E_i[i])):
        model.add( model.end_before_start(E_i[i][k - 1], E_i[i][k]) )

# Add objective
model.add( model.minimize( model.max(model.end_of(job_i) for i in E for job_i in E_i[i]) ) )

In [25]:
msol = model.solve(LogVerbosity="Normal", Workers=1, TimeLimit=30, LogPeriod=1000000,
                   log_output=True, trace_log=True, add_log_to_solution=True, RelativeOptimalityTolerance=0)

 ! --------------------------------------------------- CP Optimizer 20.1.0.0 --
 ! Minimization problem - 47 variables, 27 constraints
 ! TimeLimit            = 30
 ! Workers              = 1
 ! LogPeriod            = 1000000
 ! RelativeOptimalityTolerance = 0
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 233.3 (before), 233.3 (after)
 !  . Memory usage      : 543.6 kB (before), 543.6 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed            Branch decision
                        0         47                 -
 + New bound is 269
 *          1238       87  0.00s               (gap is 78.27%)
            1238       88         47        F        -
 + New bound is 1238 (gap is 0.00%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : 12

In [31]:
sol = csp_transform_solution(msol, M_k, types_k, instance )

In [33]:
sol.machines[0]

Machine(machine_num=0, objective=0, last_job=-1, job_schedule=[Job(id=1, start_time=0, end_time=93), Job(id=4, start_time=93, end_time=175), Job(id=17, start_time=175, end_time=269), Job(id=10, start_time=269, end_time=362), Job(id=14, start_time=362, end_time=443), Job(id=7, start_time=520, end_time=614), Job(id=12, start_time=614, end_time=706), Job(id=5, start_time=706, end_time=799), Job(id=9, start_time=799, end_time=880), Job(id=15, start_time=880, end_time=965), Job(id=16, start_time=965, end_time=1051), Job(id=3, start_time=1051, end_time=1132)])

In [21]:
solution = jm_cmax.Heuristics.shifting_bottleneck(instance)
print(solution)

KeyboardInterrupt: 

In [37]:
machine = 0
list_events = sorted([(j_id, msol[M_k[machine][i]]) for i, j_id in enumerate(types_k[machine])], key = lambda x: x[1][0])
print("\n".join(map(str,list_events)))

(1, IntervalVarValue(start=0, end=93, size=93))
(4, IntervalVarValue(start=93, end=175, size=82))
(17, IntervalVarValue(start=175, end=269, size=94))
(10, IntervalVarValue(start=269, end=362, size=93))
(14, IntervalVarValue(start=362, end=443, size=81))
(7, IntervalVarValue(start=520, end=614, size=94))
(12, IntervalVarValue(start=614, end=706, size=92))
(5, IntervalVarValue(start=706, end=799, size=93))
(9, IntervalVarValue(start=799, end=880, size=81))
(15, IntervalVarValue(start=880, end=965, size=85))
(16, IntervalVarValue(start=965, end=1051, size=86))
(3, IntervalVarValue(start=1051, end=1132, size=81))
